# Main-code for House-Amneties-Neighbourhood-Crime-Income

In [12]:
# 3rd party imports
import os
import json
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pylab as plt
from bs4 import BeautifulSoup
#!pip install pandas openpyxl


# Configure Notebook
%matplotlib inline
plt.style.use('fivethirtyeight')
sns.set_context("notebook")
import warnings
warnings.filterwarnings('ignore')
%config Completer.use_jedi = False


In [13]:
#Load CSV 
tn_df = pd.read_csv("Toronto-Neigh.csv", encoding='Windows-1252')
tnT_df=tn_df.T

# View DataFrame
tn_df.head()

,Neighbourhood Name,West Humber-Clairville,Mount Olive-Silverstone-Jamestown,Thistletown-Beaumond Heights,Rexdale-Kipling,Elms-Old Rexdale,Kingsview Village-The Westway,Willowridge-Martingrove-Richview,Humber Heights-Westmount,Edenbridge-Humber Valley,...,Harbourfront-CityPlace,St Lawrence-East Bayfront-The Islands,Church-Wellesley,Downtown Yonge East,Bay-Cloverhill,Yonge-Bay Corridor,Junction-Wallace Emerson,Dovercourt Village,North Toronto,South Eglinton-Davisville
0,Neighbourhood Number,1,2,3,4,5,6,7,8,9,...,165,166,167,168,169,170,171,172,173,174
1,TSNS 2020 Designation,Not an NIA or Emerging Neighbourhood,Neighbourhood Improvement Area,Neighbourhood Improvement Area,Not an NIA or Emerging Neighbourhood,Neighbourhood Improvement Area,Neighbourhood Improvement Area,Not an NIA or Emerging Neighbourhood,Emerging Neighbourhood,Not an NIA or Emerging Neighbourhood,...,Not an NIA or Emerging Neighbourhood,Not an NIA or Emerging Neighbourhood,Not an NIA or Emerging Neighbourhood,Not an NIA or Emerging Neighbourhood,Not an NIA or Emerging Neighbourhood,Not an NIA or Emerging Neighbourhood,Not an NIA or Emerging Neighbourhood,Not an NIA or Emerging Neighbourhood,Not an NIA or Emerging Neighbourhood,Not an NIA or Emerging Neighbourhood
2,Total - Age groups of the population - 25% sam...,33300,31345,9850,10375,9355,22005,22445,10005,15190,...,28135,31285,22320,17700,16670,12645,23180,12380,15885,22735
3,0 to 14 years,4295,5690,1495,1575,1610,3915,3500,1370,2070,...,2065,2285,895,1055,745,970,3075,1365,1315,2190
4,0 to 4 years,1460,1650,505,505,440,1245,1065,395,520,...,1030,1045,495,480,370,500,1135,445,535,910


## Filtering and Cleaning the dataframe to obtain the required data set.

In [14]:
# Extract the first data row (index 1, which is the second row of the sheet)
first_row = tn_df.T.iloc[0].astype(str)

# Keywords to search for
keywords = [
    'Income statistics in 2020 for the population aged 15 years and over in private households - 25% sample data',
   # 'Income', 
    'tax',
    'after-tax income',
    'after-tax income in 2020',
    'Median total income'
]

# Initialize a dictionary to store all indexes for each keyword
all_keyword_indexes = {}

# Find keywords in the first row (case-insensitive matching) and their corresponding indexes
for keyword in keywords:
    # Find all indexes for the keyword
    indexes = first_row[first_row.str.contains(keyword, case=False, na=False)].index.tolist()
    
    # If keyword is found, store the indexes
    if indexes:
        # Store the indexes for the keyword
        all_keyword_indexes[keyword] = indexes
        
        # Calculate and print only the min and max indexes
        min_index = min(indexes)  # Get the minimum index
        max_index = max(indexes)  # Get the maximum index
        print(f"Keyword '{keyword}' found at min index: {min_index} and max index: {max_index}")



Keyword 'Income statistics in 2020 for the population aged 15 years and over in private households - 25% sample data' found at min index: 60 and max index: 1949
Keyword 'tax' found at min index: 64 and max index: 424
Keyword 'after-tax income' found at min index: 64 and max index: 424
Keyword 'after-tax income in 2020' found at min index: 65 and max index: 166
Keyword 'Median total income' found at min index: 62 and max index: 421


In [15]:
# Access the actual values at columns 60 and 1949 (first row)
value_at_index_60 = tn_df.T.iloc[0,92]  # 61st column, first row (index 0)
value_at_index_1949 = tn_df.T.iloc[0, 424]  # 1950th column, first row (index 0)

# Optionally, get the column names at those indices
column_name_60 = tn_df.T.columns[92]  # 61st column name
column_name_1949 = tn_df.T.columns[424]  # 1950th column name

# Print the results
print(f"Value at column {column_name_60} (index 60): {value_at_index_60}")
print(f"Value at column {column_name_1949} (index 1949): {value_at_index_1949}")


Value at column 92 (index 60):     Under $10,000 (including loss)
Value at column 424 (index 1949):   Average after-tax income of one-parent economic family in 2020 ($)


In [16]:
# Step 1: Extract the first column (assuming it contains labels or identifiers)
first_column = tn_df.T.iloc[:, 0]

# Step 2: Filter the columns based on index range (between 60 and 168)
filtered_columns = tn_df.T.iloc[:, 60:167]  # Select columns from index 60 to 168 (inclusive of 60, exclusive of 169)

# Step 3: Create a new DataFrame with the first column and the filtered columns
filtered_df = pd.concat([first_column, filtered_columns], axis=1)

# Step 4: Print the resulting filtered DataFrame
(filtered_df)


,0,60,61,62,63,64,65,66,67,68,...,157,158,159,160,161,162,163,164,165,166
Neighbourhood Name,Neighbourhood Number,Total - Income statistics in 2020 for the popu...,Number of total income recipients aged 15 ye...,Median total income in 2020 among recipie...,Average total income in 2020 among recipie...,Number of after-tax income recipients aged 1...,Median after-tax income in 2020 among reci...,Average after-tax income in 2020 among rec...,Number of market income recipients aged 15 y...,Median market income in 2020 among recipi...,...,Composition of total income in 2019 of the pop...,Market income (%),Employment income (%),Government transfers (%),Employment insurance benefits (%),Total - Income statistics for persons aged 15 ...,Median total income in 2020 ($),Average total income in 2020 ($),Median after-tax income in 2020 ($),Average after-tax income in 2020 ($)
West Humber-Clairville,1,29000,27555,33600,40560,27570,31600,35800,23565,26400,...,100,83.8,75,16.2,1.4,4990,30600,37040,29200,32840
Mount Olive-Silverstone-Jamestown,2,25660,23990,29600,35000,23995,28400,31760,18275,22200,...,100,77.2,70.8,22.8,1.5,3530,28000,33920,27200,30480
Thistletown-Beaumond Heights,3,8355,7910,32800,41520,7925,30600,36360,6450,25800,...,100,82,71.6,18,1.6,1320,32800,39300,30800,34700
Rexdale-Kipling,4,8800,8445,33600,42040,8470,31800,36880,7010,28400,...,100,82.2,72.2,17.9,1,1715,27400,37040,26800,32960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yonge-Bay Corridor,170,11680,10930,44000,68200,10930,40400,53050,9985,44000,...,100,96.4,81.6,3.7,0.4,5600,40800,61600,37200,48920
Junction-Wallace Emerson,171,20100,19420,41200,54350,19420,37200,45480,16950,39600,...,100,90,83.2,10.1,0.9,5255,36800,49640,33600,42160
Dovercourt Village,172,11015,10600,38000,55850,10600,34800,46200,9525,33600,...,100,90.4,81.2,9.5,0.7,3170,34400,49080,31800,41560
North Toronto,173,14570,13865,46000,58100,13865,41200,48200,12585,44400,...,100,92.8,81.6,7.3,0.6,5935,48400,59700,42800,49160


In [17]:
# Step 1: Reset the row index and add it as a column (this step you already have)
filtered_df_reset = filtered_df.reset_index()

# Step 2: Set the row index column to the first column (this you already did)
filtered_df_reset.set_index(filtered_df_reset.columns[0], inplace=True)

# Step 3: Reset column headers to default (0, 1, 2, ...)
# This step will reset the column headers (the numeric indices you mentioned)
filtered_df_reset.columns = range(filtered_df_reset.shape[1])

# Confirmation message
print("Row index has been moved to the first column, and column headers have been reset.")
filtered_df_reset

Row index has been moved to the first column, and column headers have been reset.


,0,1,2,3,4,5,6,7,8,9,...,98,99,100,101,102,103,104,105,106,107
index,,,,,,,,,,,,,,,,,,,,,
Neighbourhood Name,Neighbourhood Number,Total - Income statistics in 2020 for the popu...,Number of total income recipients aged 15 ye...,Median total income in 2020 among recipie...,Average total income in 2020 among recipie...,Number of after-tax income recipients aged 1...,Median after-tax income in 2020 among reci...,Average after-tax income in 2020 among rec...,Number of market income recipients aged 15 y...,Median market income in 2020 among recipi...,...,Composition of total income in 2019 of the pop...,Market income (%),Employment income (%),Government transfers (%),Employment insurance benefits (%),Total - Income statistics for persons aged 15 ...,Median total income in 2020 ($),Average total income in 2020 ($),Median after-tax income in 2020 ($),Average after-tax income in 2020 ($)
West Humber-Clairville,1,29000,27555,33600,40560,27570,31600,35800,23565,26400,...,100,83.8,75,16.2,1.4,4990,30600,37040,29200,32840
Mount Olive-Silverstone-Jamestown,2,25660,23990,29600,35000,23995,28400,31760,18275,22200,...,100,77.2,70.8,22.8,1.5,3530,28000,33920,27200,30480
Thistletown-Beaumond Heights,3,8355,7910,32800,41520,7925,30600,36360,6450,25800,...,100,82,71.6,18,1.6,1320,32800,39300,30800,34700
Rexdale-Kipling,4,8800,8445,33600,42040,8470,31800,36880,7010,28400,...,100,82.2,72.2,17.9,1,1715,27400,37040,26800,32960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yonge-Bay Corridor,170,11680,10930,44000,68200,10930,40400,53050,9985,44000,...,100,96.4,81.6,3.7,0.4,5600,40800,61600,37200,48920
Junction-Wallace Emerson,171,20100,19420,41200,54350,19420,37200,45480,16950,39600,...,100,90,83.2,10.1,0.9,5255,36800,49640,33600,42160
Dovercourt Village,172,11015,10600,38000,55850,10600,34800,46200,9525,33600,...,100,90.4,81.2,9.5,0.7,3170,34400,49080,31800,41560


In [18]:
# Step 1: Reset the index, which makes the current index a column
filtered_df_reset1 = filtered_df_reset.reset_index()

# Optional Step 2: Set a specific column as the new index (if you need a specific index)
# df_reset.set_index('your_column_name', inplace=True)

# Confirmation message
print("Index column has been moved to the first column, and row index has been reset.")

filtered_df_reset1= filtered_df_reset1.rename(columns={"index": "Neighbourhood"})
filtered_df_reset1

Index column has been moved to the first column, and row index has been reset.


,Neighbourhood,0,1,2,3,4,5,6,7,8,...,98,99,100,101,102,103,104,105,106,107
0,Neighbourhood Name,Neighbourhood Number,Total - Income statistics in 2020 for the popu...,Number of total income recipients aged 15 ye...,Median total income in 2020 among recipie...,Average total income in 2020 among recipie...,Number of after-tax income recipients aged 1...,Median after-tax income in 2020 among reci...,Average after-tax income in 2020 among rec...,Number of market income recipients aged 15 y...,...,Composition of total income in 2019 of the pop...,Market income (%),Employment income (%),Government transfers (%),Employment insurance benefits (%),Total - Income statistics for persons aged 15 ...,Median total income in 2020 ($),Average total income in 2020 ($),Median after-tax income in 2020 ($),Average after-tax income in 2020 ($)
1,West Humber-Clairville,1,29000,27555,33600,40560,27570,31600,35800,23565,...,100,83.8,75,16.2,1.4,4990,30600,37040,29200,32840
2,Mount Olive-Silverstone-Jamestown,2,25660,23990,29600,35000,23995,28400,31760,18275,...,100,77.2,70.8,22.8,1.5,3530,28000,33920,27200,30480
3,Thistletown-Beaumond Heights,3,8355,7910,32800,41520,7925,30600,36360,6450,...,100,82,71.6,18,1.6,1320,32800,39300,30800,34700
4,Rexdale-Kipling,4,8800,8445,33600,42040,8470,31800,36880,7010,...,100,82.2,72.2,17.9,1,1715,27400,37040,26800,32960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,Yonge-Bay Corridor,170,11680,10930,44000,68200,10930,40400,53050,9985,...,100,96.4,81.6,3.7,0.4,5600,40800,61600,37200,48920
155,Junction-Wallace Emerson,171,20100,19420,41200,54350,19420,37200,45480,16950,...,100,90,83.2,10.1,0.9,5255,36800,49640,33600,42160
156,Dovercourt Village,172,11015,10600,38000,55850,10600,34800,46200,9525,...,100,90.4,81.2,9.5,0.7,3170,34400,49080,31800,41560
157,North Toronto,173,14570,13865,46000,58100,13865,41200,48200,12585,...,100,92.8,81.6,7.3,0.6,5935,48400,59700,42800,49160


## Loading the file from the previous pipeline step.

In [19]:
#second file
hos_t=pd.read_csv("Toronto_houses_with_crime_data_1km.csv")
hos_t

,Unnamed: 0,Street address,Zip/Postal Code,Latitude,Longitude,Region,Price,Bedrooms,Bathrooms,Square Footage,...,geometry,Neighbourhood,Total_crimes (2020-2024),Average_total_crime (per_year),Total_severe_crimes (2020-2024),Average_severe_crimes (per_year),Total_moderate_crimes (2020-2024),Average_moderate_crimes (per_year),Total_minor_crimes (2020-2024),Average_minor_crimes (per_year)
0,0,1787 Saint Clair Ave W #414,M6N1H9,43.673553,-79.461868,York,525000.0,2 bds,1 ba,NaN,...,POINT (-79.461868 43.673553),Weston-Pelham Park,910.0,182.0,521.0,104.0,263.0,53.0,126.0,25.0
1,1,509 Beecroft Rd #1209,M2N0A3,43.778652,-79.417947,North York,848000.0,3 bds,2 ba,NaN,...,POINT (-79.417947 43.778652),Willowdale West,978.0,196.0,329.0,66.0,363.0,73.0,286.0,57.0
2,3,186 Macpherson Ave,M5R1W8,43.677945,-79.396728,Central Toronto,2218000.0,3 bds,3 ba,NaN,...,POINT (-79.396728 43.677945),Annex,3265.0,653.0,1417.0,283.0,1116.0,223.0,732.0,146.0
3,4,195 McCaul St #909,M5T0E5,43.656700,-79.392206,Downtown Toronto,449900.0,Studio,1 ba,NaN,...,POINT (-79.392206 43.6567),Kensington-Chinatown,3837.0,767.0,1960.0,392.0,996.0,199.0,881.0,176.0
4,5,30 Roehampton Ave UNIT 1902,M4P0B9,43.708432,-79.397532,Central Toronto,630000.0,2 bds,2 ba,NaN,...,POINT (-79.397532 43.708432),North Toronto,988.0,198.0,487.0,97.0,343.0,69.0,158.0,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133,1150,804 - 3 Marine Parade Dr,NaN,43.630233,-79.476220,NaN,675000.0,2,2,NaN,...,POINT (-79.47622 43.630233),Humber Bay Shores,1174.0,235.0,572.0,114.0,375.0,75.0,227.0,45.0
1134,1151,1811 - 501 St Clair Ave W,NaN,43.683010,-79.417709,NaN,848888.0,2,2,NaN,...,POINT (-79.417709 43.68301),Casa Loma,765.0,153.0,273.0,55.0,357.0,71.0,135.0,27.0
1135,1152,215 Searle Ave,NaN,43.756634,-79.456582,NaN,1789000.0,5,2,NaN,...,POINT (-79.456582 43.756634),Bathurst Manor,1293.0,259.0,409.0,82.0,548.0,110.0,336.0,67.0
1136,1153,603 - 430 Roncesvalles Ave,NaN,43.651984,-79.451509,NaN,799000.0,2,2,NaN,...,POINT (-79.451509 43.651984),High Park-Swansea,1307.0,261.0,475.0,95.0,561.0,112.0,271.0,54.0


### Combining the data_sets

In [20]:
df=hos_t
df['Neighbourhood'] = df['Neighbourhood'].str.lower()
column_name = 'Neighbourhood'

# Get unique entries in the column
unique_entries = df[column_name].dropna().unique()

# Convert to a list if needed (or you can leave it as an array)
unique_entries_list = unique_entries.tolist()

# Display the unique entries
len(unique_entries_list)

152

In [21]:
df=filtered_df_reset1
df['Neighbourhood'] = df['Neighbourhood'].str.lower()
column_name = 'Neighbourhood'

# Get unique entries in the column
unique_entries = df[column_name].dropna().unique()

# Convert to a list if needed (or you can leave it as an array)
unique_entries_list = unique_entries.tolist()

# Display the unique entries
len(unique_entries_list)


159

In [22]:
# Load the two DataFrames
df1 = hos_t
df2 = filtered_df_reset1

# Specify the column with unique entries in df1 and the corresponding column in df2
unique_column_df1 = 'Neighbourhood'
matching_column_df2 = 'Neighbourhood'

# Clean the 'Neighbourhood' column in both DataFrames
#df1[unique_column_df1] = df1[unique_column_df1].str.strip().str.lower().replace({'\'': '', ',': '': ''}, regex=True)
df1[unique_column_df1] = df1[unique_column_df1].str.strip().str.lower().replace({'\'': ' ', ',': ' ', '-': ' ','`': ' '}, regex=True).str.replace(r'\s+', ' ', regex=True)

df2[matching_column_df2] = df2[matching_column_df2].str.strip().str.lower().replace({'\'': ' ', ',': ' ', '-': ' ','`': ' '}, regex=True).str.replace(r'\s+', ' ', regex=True)

df1[unique_column_df1] = df1[unique_column_df1].str.replace(r'\bst\.\b', 'st ', regex=True, case=False)
df2[matching_column_df2] = df2[matching_column_df2].str.replace(r'st\.', 'st ', regex=True, case=False)

df1[unique_column_df1] = df1[unique_column_df1].str.replace(r'\s+', ' ', regex=True)
df2[matching_column_df2] = df2[matching_column_df2].str.replace(r'\s+', ' ', regex=True)


# Step 1: Get unique entries from df1 (now cleaned)
unique_entries = df1[unique_column_df1].dropna().unique()

# Step 2: Filter rows in df2 where the matching column values are in unique_entries
filtered_df2 = df2[df2[matching_column_df2].isin(unique_entries)]

# Display the filtered dataframe
filtered_df2


,Neighbourhood,0,1,2,3,4,5,6,7,8,...,98,99,100,101,102,103,104,105,106,107
1,west humber clairville,1,29000,27555,33600,40560,27570,31600,35800,23565,...,100,83.8,75,16.2,1.4,4990,30600,37040,29200,32840
2,mount olive silverstone jamestown,2,25660,23990,29600,35000,23995,28400,31760,18275,...,100,77.2,70.8,22.8,1.5,3530,28000,33920,27200,30480
4,rexdale kipling,4,8800,8445,33600,42040,8470,31800,36880,7010,...,100,82.2,72.2,17.9,1,1715,27400,37040,26800,32960
5,elms old rexdale,5,7745,7325,34400,41240,7325,32400,36440,6015,...,100,81.2,72.2,18.9,1.3,1000,33600,40200,31200,35300
6,kingsview village the westway,6,18090,17240,34400,44880,17240,32400,38640,14225,...,100,82,67.6,18.1,1.1,3080,34400,44800,32400,38360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,yonge bay corridor,170,11680,10930,44000,68200,10930,40400,53050,9985,...,100,96.4,81.6,3.7,0.4,5600,40800,61600,37200,48920
155,junction wallace emerson,171,20100,19420,41200,54350,19420,37200,45480,16950,...,100,90,83.2,10.1,0.9,5255,36800,49640,33600,42160
156,dovercourt village,172,11015,10600,38000,55850,10600,34800,46200,9525,...,100,90.4,81.2,9.5,0.7,3170,34400,49080,31800,41560
157,north toronto,173,14570,13865,46000,58100,13865,41200,48200,12585,...,100,92.8,81.6,7.3,0.6,5935,48400,59700,42800,49160


In [23]:
# Get entries in df1 that are not in df2
unmatched_entries = set(unique_entries) - set(df2[matching_column_df2].unique())
print("Unmatched entries:", unmatched_entries)


Unmatched entries: set()


### Getting only the subset of income dataset

In [24]:
# Select the specific row in df1 that you want to use as column headers
# For example, if it's the first row (index 0):
df1= filtered_df_reset
new_headers = df1.iloc[0]
len(new_headers)

df2= filtered_df2
# # Update df2's column names with these new headers
# Keep the first column header of df2 unchanged
df2_columns = [df2.columns[0]] + list(new_headers)

# Update df2's columns
df2.columns = df2_columns

# # Confirm the change
# print("Updated column names of df2:")
income_1=df2
income_1



,Neighbourhood,Neighbourhood Number,Total - Income statistics in 2020 for the population aged 15 years and over in private households - 25% sample data,Number of total income recipients aged 15 years and over in private households in 2020 - 25% sample data,Median total income in 2020 among recipients ($),Average total income in 2020 among recipients ($),Number of after-tax income recipients aged 15 years and over in private households in 2020 - 25% sample data,Median after-tax income in 2020 among recipients ($),Average after-tax income in 2020 among recipients ($),Number of market income recipients aged 15 years and over in private households in 2020 - 25% sample data,...,Composition of total income in 2019 of the population aged 15 years and over in private households (%) - 25% sample data,Market income (%),Employment income (%),Government transfers (%),Employment insurance benefits (%),Total - Income statistics for persons aged 15 years and over not in economic families in private households - 25% sample data,Median total income in 2020 ($),Average total income in 2020 ($),Median after-tax income in 2020 ($),Average after-tax income in 2020 ($)
1,west humber clairville,1,29000,27555,33600,40560,27570,31600,35800,23565,...,100,83.8,75,16.2,1.4,4990,30600,37040,29200,32840
2,mount olive silverstone jamestown,2,25660,23990,29600,35000,23995,28400,31760,18275,...,100,77.2,70.8,22.8,1.5,3530,28000,33920,27200,30480
4,rexdale kipling,4,8800,8445,33600,42040,8470,31800,36880,7010,...,100,82.2,72.2,17.9,1,1715,27400,37040,26800,32960
5,elms old rexdale,5,7745,7325,34400,41240,7325,32400,36440,6015,...,100,81.2,72.2,18.9,1.3,1000,33600,40200,31200,35300
6,kingsview village the westway,6,18090,17240,34400,44880,17240,32400,38640,14225,...,100,82,67.6,18.1,1.1,3080,34400,44800,32400,38360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,yonge bay corridor,170,11680,10930,44000,68200,10930,40400,53050,9985,...,100,96.4,81.6,3.7,0.4,5600,40800,61600,37200,48920
155,junction wallace emerson,171,20100,19420,41200,54350,19420,37200,45480,16950,...,100,90,83.2,10.1,0.9,5255,36800,49640,33600,42160
156,dovercourt village,172,11015,10600,38000,55850,10600,34800,46200,9525,...,100,90.4,81.2,9.5,0.7,3170,34400,49080,31800,41560
157,north toronto,173,14570,13865,46000,58100,13865,41200,48200,12585,...,100,92.8,81.6,7.3,0.6,5935,48400,59700,42800,49160


In [25]:
# Select and convert the necessary columns to lists
columns_to_keep = ['Neighbourhood'] + ['    Average total income in 2020 among recipients ($)'] + ['    Average after-tax income in 2020 among recipients ($)'] #+ income_1.iloc[:,31:80]

# Use the columns_to_keep list to select those columns from the DataFrame
income = income_1[columns_to_keep]

# Display the new DataFrame
(income)


,Neighbourhood,Average total income in 2020 among recipients ($),Average after-tax income in 2020 among recipients ($)
1,west humber clairville,40560,35800
2,mount olive silverstone jamestown,35000,31760
4,rexdale kipling,42040,36880
5,elms old rexdale,41240,36440
6,kingsview village the westway,44880,38640
...,...,...,...
154,yonge bay corridor,68200,53050
155,junction wallace emerson,54350,45480
156,dovercourt village,55850,46200
157,north toronto,58100,48200


In [26]:
# first_column = income_2[['Neighbourhood']]
# sec_col = income_2[['    Average total income in 2020 among recipients ($)']]
# third_col =income_2[['    Average after-tax income in 2020 among recipients ($)']]

# Step 2: Split the remaining columns into two groups
# For example, we will split the columns based on indices
# You can adjust the column indices as per your requirement

# Group 1: Columns 1 to N (excluding the first column)
# Adjust the range of columns as necessary
df_1 = income_1.iloc[:, 31:47]  # First half of remaining columns
df_2 = income_1.iloc[:, 47:63]   # Second half of remaining columns
df_3 = income_1.iloc[:, 63:80]   # Second half of remaining columns

# # Step 3: Add the 'Neigh' column back to each new DataFrame
df_1 = pd.concat([income,df_1], axis=1) #gross
df_2 = pd.concat([income,df_2], axis=1) #after-tax
df_3 = pd.concat([income,df_3],axis=1)  #employement


In [27]:
#### Combining the income braackets to define 6 classes

In [28]:
df_2.iloc[:,7]

1      4370
2      4180
4      1280
5      1100
6      2290
       ... 
154    1145
155    2410
156    1300
157    1380
158    1800
Name:     $10,000 to $19,999, Length: 152, dtype: object

In [29]:
##Gross-income of individuals

# Add the values of two columns (e.g., 'column1' and 'column2') and assign the result to a new column
avg_expense = 1530*12

df_1['Class 6G'] = pd.to_numeric(df_1.iloc[:,7], errors='coerce') + pd.to_numeric(df_1.iloc[:,8], errors='coerce')
df_1['Class 5G'] = pd.to_numeric(df_1.iloc[:,9], errors='coerce') + pd.to_numeric(df_1.iloc[:,10], errors='coerce') +pd.to_numeric(df_1.iloc[:,11], errors='coerce')
df_1['Class 4G'] = pd.to_numeric(df_1.iloc[:,12], errors='coerce') + pd.to_numeric(df_1.iloc[:,13], errors='coerce') 
df_1['Class 3G'] = pd.to_numeric(df_1.iloc[:,14], errors='coerce') + pd.to_numeric(df_1.iloc[:,15], errors='coerce') +pd.to_numeric(df_1.iloc[:,16], errors='coerce')
df_1['Class 2G'] = pd.to_numeric(df_1.iloc[:,17], errors='coerce') 
df_1['Class 1G'] = pd.to_numeric(df_1.iloc[:,18], errors='coerce') 


# Print the DataFrame to confirm the new column
(df_1)


,Neighbourhood,Average total income in 2020 among recipients ($),Average after-tax income in 2020 among recipients ($),Total - Total income groups in 2020 for the population aged 15 years and over in private households - 25% sample data,Without total income,With total income,"Under $10,000 (including loss)","$10,000 to $19,999","$20,000 to $29,999","$30,000 to $39,999",...,"$90,000 to $99,999","$100,000 and over","$100,000 to $149,999","$150,000 and over",Class 6G,Class 5G,Class 4G,Class 3G,Class 2G,Class 1G
1,west humber clairville,40560,35800,29000,1445,27555,2600,4155,5285,4260,...,450,1175,930,240,9440,10115,2935,2465,930,240
2,mount olive silverstone jamestown,35000,31760,25660,1665,23990,2875,4035,5280,3735,...,305,495,415,80,9315,8455,2065,1285,415,80
4,rexdale kipling,42040,36880,8800,355,8445,680,1200,1850,1150,...,200,470,375,95,3050,2860,930,935,375,95
5,elms old rexdale,41240,36440,7745,420,7325,705,1090,1375,1080,...,155,340,275,65,2465,2655,735,775,275,65
6,kingsview village the westway,44880,38640,18090,855,17240,1680,2195,3470,2730,...,395,1230,875,350,5665,6075,1665,2160,875,350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,yonge bay corridor,68200,53050,11680,750,10930,1540,1090,1375,1080,...,510,1820,1005,820,2465,2625,1420,2870,1005,820
155,junction wallace emerson,54350,45480,20100,680,19420,1440,2290,3310,2475,...,665,2490,1755,735,5600,5845,2470,4065,1755,735
156,dovercourt village,55850,46200,11015,420,10600,825,1235,2000,1485,...,315,1475,910,560,3235,3205,1130,2215,910,560
157,north toronto,58100,48200,14570,705,13865,1185,1310,1945,1665,...,605,1800,1265,535,3255,4280,1960,3190,1265,535


In [30]:
## After tax reduction income

# Add the values of two columns (e.g., 'column1' and 'column2') and assign the result to a new column
avg_expense = 1530

df_2['Class 6G_T'] = pd.to_numeric(df_2.iloc[:,7]) + pd.to_numeric(df_2.iloc[:,8])
df_2['Class 5G_T'] = pd.to_numeric(df_2.iloc[:,9]) + pd.to_numeric(df_2.iloc[:,10]) +pd.to_numeric(df_2.iloc[:,11])
df_2['Class 4G_T'] = pd.to_numeric(df_2.iloc[:,12]) + pd.to_numeric(df_2.iloc[:,13]) 
df_2['Class 3G_T'] = pd.to_numeric(df_2.iloc[:,14]) + pd.to_numeric(df_2.iloc[:,15]) +pd.to_numeric(df_2.iloc[:,16])
df_2['Class 2G_T'] = pd.to_numeric(df_2.iloc[:,17]) 
df_2['Class 1G_T'] = pd.to_numeric(df_2.iloc[:,18]) 

# Print the DataFrame to confirm the new column
(df_2)


,Neighbourhood,Average total income in 2020 among recipients ($),Average after-tax income in 2020 among recipients ($),Total - After-tax income groups in 2020 for the population aged 15 years and over in private households - 25% sample data,Without after-tax income,With after-tax income,"Under $10,000 (including loss)","$10,000 to $19,999","$20,000 to $29,999","$30,000 to $39,999",...,"$90,000 to $99,999","$100,000 and over","$100,000 to $124,999","$125,000 and over",Class 6G_T,Class 5G_T,Class 4G_T,Class 3G_T,Class 2G_T,Class 1G_T
1,west humber clairville,40560,35800,29000,1430,27570,2650,4370,5900,4945,...,340,450,295,155,10270,10960,2425,1260,295,155
2,mount olive silverstone jamestown,35000,31760,25660,1660,23995,2905,4180,5825,4285,...,145,175,105,70,10005,9025,1450,610,105,70
4,rexdale kipling,42040,36880,8800,330,8470,710,1280,2000,1315,...,110,150,70,70,3280,3200,780,495,70,70
5,elms old rexdale,41240,36440,7745,420,7325,715,1100,1590,1260,...,90,135,85,45,2690,2820,725,385,85,45
6,kingsview village the westway,44880,38640,18090,855,17240,1705,2290,3890,2865,...,295,540,285,255,6180,6480,1575,1305,285,255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,yonge bay corridor,68200,53050,11680,750,10930,1575,1145,1550,1150,...,240,1100,450,645,2695,3105,1630,1930,450,645
155,junction wallace emerson,54350,45480,20100,680,19420,1445,2410,3715,2870,...,565,1205,755,455,6125,6665,2615,2560,755,455
156,dovercourt village,55850,46200,11015,420,10600,845,1300,2350,1485,...,220,870,415,450,3650,3445,1160,1500,415,450
157,north toronto,58100,48200,14570,705,13865,1205,1380,2325,1840,...,410,830,460,370,3705,4980,2115,1865,460,370


In [31]:
# Step 1: Ensure "Neighbourhood" is the index in both DataFrames
df_1.set_index('Neighbourhood', inplace=True)  # Modify df_1 in place
hos_t.set_index('Neighbourhood', inplace=True)  # Modify hos_t in place
# df_1.set_index('Neighbourhood', inplace= False)
# hos_t.set_index('Neighbourhood', inplace= False)

# Step 2: Select the last 6 columns from df_1
last_6_columns_df1 = df_1.iloc[:, -6:]

# Step 3: Merge these columns into hos_t based on the "Neighbourhood" index
hos_t = hos_t.merge(last_6_columns_df1, how='left', left_index=True, right_index=True)

# Step 4: Reset the index if you want the "Neighbourhood" back as a column
hos_t.reset_index(inplace=True)

# Step 5: Display the updated hos_t DataFrame
(hos_t)


,Neighbourhood,Unnamed: 0,Street address,Zip/Postal Code,Latitude,Longitude,Region,Price,Bedrooms,Bathrooms,...,Total_moderate_crimes (2020-2024),Average_moderate_crimes (per_year),Total_minor_crimes (2020-2024),Average_minor_crimes (per_year),Class 6G,Class 5G,Class 4G,Class 3G,Class 2G,Class 1G
0,agincourt north,639,435 Middlefield Rd #56,M1S5W1,43.808839,-79.256579,Scarborough,848800.0,4 bds,4 ba,...,689.0,138.0,248.0,50.0,9005.0,7485.0,1960.0,2020.0,780.0,175.0
1,agincourt north,755,78 Ashridge Dr,NaN,43.815736,-79.268628,NaN,998000.0,5,3,...,689.0,138.0,248.0,50.0,9005.0,7485.0,1960.0,2020.0,780.0,175.0
2,agincourt north,873,48 Ardgowan Cres,NaN,43.812243,-79.271403,NaN,898000.0,4,3,...,689.0,138.0,248.0,50.0,9005.0,7485.0,1960.0,2020.0,780.0,175.0
3,agincourt north,1082,119 - 5235 Finch Ave E,NaN,43.809932,-79.254877,NaN,560000.0,3,2,...,689.0,138.0,248.0,50.0,9005.0,7485.0,1960.0,2020.0,780.0,175.0
4,agincourt south malvern west,477,60 Manorglen Cres,M1S1W4,43.780125,-79.271026,Scarborough,1199000.0,4 bds,3 ba,...,778.0,156.0,372.0,74.0,7035.0,6535.0,2085.0,2160.0,800.0,270.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133,NaN,761,Th5 - 118 Finch Avenue,NaN,43.833506,-79.152600,NaN,899000.0,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1134,NaN,964,18 - 73 Driftwood Ave,NaN,43.940271,-79.468319,NaN,618000.0,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1135,NaN,1052,1 Versailles Crt,NaN,43.879039,-79.431228,NaN,4288888.0,6,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1136,NaN,1063,Ph 3 - 850 Steeles Ave W,NaN,43.777253,-79.514493,NaN,1198900.0,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# Drop rows that have NaN in specific columns 
final_df = hos_t.dropna(subset=['Class 6G'])

# Display the DataFrame after dropping rows
(final_df)


,Neighbourhood,Unnamed: 0,Street address,Zip/Postal Code,Latitude,Longitude,Region,Price,Bedrooms,Bathrooms,...,Total_moderate_crimes (2020-2024),Average_moderate_crimes (per_year),Total_minor_crimes (2020-2024),Average_minor_crimes (per_year),Class 6G,Class 5G,Class 4G,Class 3G,Class 2G,Class 1G
0,agincourt north,639,435 Middlefield Rd #56,M1S5W1,43.808839,-79.256579,Scarborough,848800.0,4 bds,4 ba,...,689.0,138.0,248.0,50.0,9005.0,7485.0,1960.0,2020.0,780.0,175.0
1,agincourt north,755,78 Ashridge Dr,NaN,43.815736,-79.268628,NaN,998000.0,5,3,...,689.0,138.0,248.0,50.0,9005.0,7485.0,1960.0,2020.0,780.0,175.0
2,agincourt north,873,48 Ardgowan Cres,NaN,43.812243,-79.271403,NaN,898000.0,4,3,...,689.0,138.0,248.0,50.0,9005.0,7485.0,1960.0,2020.0,780.0,175.0
3,agincourt north,1082,119 - 5235 Finch Ave E,NaN,43.809932,-79.254877,NaN,560000.0,3,2,...,689.0,138.0,248.0,50.0,9005.0,7485.0,1960.0,2020.0,780.0,175.0
4,agincourt south malvern west,477,60 Manorglen Cres,M1S1W4,43.780125,-79.271026,Scarborough,1199000.0,4 bds,3 ba,...,778.0,156.0,372.0,74.0,7035.0,6535.0,2085.0,2160.0,800.0,270.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,yorkdale glen park,576,1 Bolingbroke Rd,M6B3L5,43.712749,-79.448179,North York,1625000.0,3 bds,3 ba,...,1270.0,254.0,720.0,144.0,4620.0,4375.0,1645.0,2080.0,880.0,270.0
1129,yorkdale glen park,913,354 Ranee Ave,NaN,43.722078,-79.455298,NaN,2688000.0,5,5,...,1270.0,254.0,720.0,144.0,4620.0,4375.0,1645.0,2080.0,880.0,270.0
1130,yorkdale glen park,998,61 Dane Ave,NaN,43.717200,-79.449548,NaN,3299000.0,9,5,...,1270.0,254.0,720.0,144.0,4620.0,4375.0,1645.0,2080.0,880.0,270.0
1131,yorkdale glen park,1025,374 Hillmount Ave,NaN,43.706844,-79.450609,NaN,1800000.0,9,4,...,1270.0,254.0,720.0,144.0,4620.0,4375.0,1645.0,2080.0,880.0,270.0


In [33]:
# Step 1: Ensure "Neighbourhood" is the index in both DataFrames
df_2.set_index('Neighbourhood', inplace=True)  # Modify df_1 in place
hos_t.set_index('Neighbourhood', inplace=True)  # Modify hos_t in place
# df_1.set_index('Neighbourhood', inplace= False)
# hos_t.set_index('Neighbourhood', inplace= False)

# Step 2: Select the last 6 columns from df_1
last_6_columns_df2 = df_2.iloc[:, -6:]

# Step 3: Merge these columns into hos_t based on the "Neighbourhood" index
hos_t = hos_t.merge(last_6_columns_df2, how='left', left_index=True, right_index=True)

# Step 4: Reset the index if you want the "Neighbourhood" back as a column
hos_t.reset_index(inplace=True)

# Step 5: Display the updated hos_t DataFrame
(hos_t)


,Neighbourhood,Unnamed: 0,Street address,Zip/Postal Code,Latitude,Longitude,Region,Price,Bedrooms,Bathrooms,...,Class 4G,Class 3G,Class 2G,Class 1G,Class 6G_T,Class 5G_T,Class 4G_T,Class 3G_T,Class 2G_T,Class 1G_T
0,agincourt north,639,435 Middlefield Rd #56,M1S5W1,43.808839,-79.256579,Scarborough,848800.0,4 bds,4 ba,...,1960.0,2020.0,780.0,175.0,9720.0,7890.0,1760.0,1060.0,225.0,110.0
1,agincourt north,755,78 Ashridge Dr,NaN,43.815736,-79.268628,NaN,998000.0,5,3,...,1960.0,2020.0,780.0,175.0,9720.0,7890.0,1760.0,1060.0,225.0,110.0
2,agincourt north,873,48 Ardgowan Cres,NaN,43.812243,-79.271403,NaN,898000.0,4,3,...,1960.0,2020.0,780.0,175.0,9720.0,7890.0,1760.0,1060.0,225.0,110.0
3,agincourt north,1082,119 - 5235 Finch Ave E,NaN,43.809932,-79.254877,NaN,560000.0,3,2,...,1960.0,2020.0,780.0,175.0,9720.0,7890.0,1760.0,1060.0,225.0,110.0
4,agincourt south malvern west,477,60 Manorglen Cres,M1S1W4,43.780125,-79.271026,Scarborough,1199000.0,4 bds,3 ba,...,2085.0,2160.0,800.0,270.0,7600.0,7095.0,1880.0,1160.0,265.0,175.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133,NaN,761,Th5 - 118 Finch Avenue,NaN,43.833506,-79.152600,NaN,899000.0,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1134,NaN,964,18 - 73 Driftwood Ave,NaN,43.940271,-79.468319,NaN,618000.0,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1135,NaN,1052,1 Versailles Crt,NaN,43.879039,-79.431228,NaN,4288888.0,6,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1136,NaN,1063,Ph 3 - 850 Steeles Ave W,NaN,43.777253,-79.514493,NaN,1198900.0,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
# Drop rows that have NaN in specific columns (e.g., 'Column1', 'Column2', ...)
final_df2 = hos_t.dropna(subset=['Class 6G_T'])

# Display the DataFrame after dropping rows
(final_df2)


,Neighbourhood,Unnamed: 0,Street address,Zip/Postal Code,Latitude,Longitude,Region,Price,Bedrooms,Bathrooms,...,Class 4G,Class 3G,Class 2G,Class 1G,Class 6G_T,Class 5G_T,Class 4G_T,Class 3G_T,Class 2G_T,Class 1G_T
0,agincourt north,639,435 Middlefield Rd #56,M1S5W1,43.808839,-79.256579,Scarborough,848800.0,4 bds,4 ba,...,1960.0,2020.0,780.0,175.0,9720.0,7890.0,1760.0,1060.0,225.0,110.0
1,agincourt north,755,78 Ashridge Dr,NaN,43.815736,-79.268628,NaN,998000.0,5,3,...,1960.0,2020.0,780.0,175.0,9720.0,7890.0,1760.0,1060.0,225.0,110.0
2,agincourt north,873,48 Ardgowan Cres,NaN,43.812243,-79.271403,NaN,898000.0,4,3,...,1960.0,2020.0,780.0,175.0,9720.0,7890.0,1760.0,1060.0,225.0,110.0
3,agincourt north,1082,119 - 5235 Finch Ave E,NaN,43.809932,-79.254877,NaN,560000.0,3,2,...,1960.0,2020.0,780.0,175.0,9720.0,7890.0,1760.0,1060.0,225.0,110.0
4,agincourt south malvern west,477,60 Manorglen Cres,M1S1W4,43.780125,-79.271026,Scarborough,1199000.0,4 bds,3 ba,...,2085.0,2160.0,800.0,270.0,7600.0,7095.0,1880.0,1160.0,265.0,175.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,yorkdale glen park,576,1 Bolingbroke Rd,M6B3L5,43.712749,-79.448179,North York,1625000.0,3 bds,3 ba,...,1645.0,2080.0,880.0,270.0,4960.0,5065.0,1475.0,1200.0,220.0,215.0
1129,yorkdale glen park,913,354 Ranee Ave,NaN,43.722078,-79.455298,NaN,2688000.0,5,5,...,1645.0,2080.0,880.0,270.0,4960.0,5065.0,1475.0,1200.0,220.0,215.0
1130,yorkdale glen park,998,61 Dane Ave,NaN,43.717200,-79.449548,NaN,3299000.0,9,5,...,1645.0,2080.0,880.0,270.0,4960.0,5065.0,1475.0,1200.0,220.0,215.0
1131,yorkdale glen park,1025,374 Hillmount Ave,NaN,43.706844,-79.450609,NaN,1800000.0,9,4,...,1645.0,2080.0,880.0,270.0,4960.0,5065.0,1475.0,1200.0,220.0,215.0


In [36]:
final_df2.to_csv('Toronto_houses_with_crime_income_data_1.0km.csv', index=False)